In [1]:
import numpy as np

In [2]:
'''
Convolution class using no padding

param func - activation function
param d_func - derivative of activation function
param last_layer - point to last layer, which pass the value over
param input_num - numbers of input feature maps/images 
param input_size - input feature maps/images size
param filter_size - size of the filter/kernel
param filter_num - numbers of filters, refer to number of output feature maps/images 
param stride - moving step of the kernel function
param is_first - whether this layer is the first layer
'''
class ConvLayer():
    def __init__(self, func, d_func, last_layer, input_size, input_num, filter_size, filter_num, stride, is_first):
        # Activation Functions
        self.act_func = func                                      # Activation function
        self.d_act_func = d_func                                  # Derivative of activate function
        
        self.last_layer = last_layer
        self.input_num = int(input_num)
        self.input_size = int(input_size)
        self.filter_size = int(filter_size)
        self.filter_num = int(filter_num)
        self.stride = int(stride)
        self.is_first = is_first
        
        # Initial kernel
        bound = np.sqrt(6 / ((input_num + filter_num) * filter_size * filter_size))
        self.kernel = np.random.uniform(-bound, bound, (int(filter_num),int(input_num),int(filter_size),int(filter_size)))
#         self.kernel = np.random.randn(int(filter_num),int(input_num),int(filter_size),int(filter_size))
#         self.kernel = np.random.randn(int(filter_num),int(input_num),int(filter_size),int(filter_size)) / np.sqrt(1/(filter_size*filter_size)
        # Initial bias
        self.bias = np.zeros(self.filter_num)
        
        # Check if the parameters of input size, filter size, stride is legel
        self.output_size = (input_size + stride - filter_size) / stride
        if(self.output_size%1 != 0):
            print("Invalid ! Please check your parameters");
            return -1
        
        self.input_img = 0
        self.stride_shape = (int(input_num),int(self.output_size),int(self.output_size),int(filter_size),int(filter_size))
        self.strides = (int(input_size*input_size*8), int(input_size*stride*8), int(stride*8), int(input_size*8), 8)
        self.conv_img = 0
        self.output_img = 0
        self.d_func_img = 0
        
        self.delta_bias = 0      # Correction of bias
        self.pass_error = 0      # Error passed to previous layer
        self.delta_w = 0         # Weight correction
        self.error = 0           # input error * derivative of activation function
        
    def forwrad_pass(self):
        if not self.is_first:
            self.extract_value()
        self.strided_img = np.lib.stride_tricks.as_strided(self.input_img, shape=self.stride_shape, strides=self.strides)       # Cut the image with kernel size
        
        # Convolution operations
        self.conv_img = np.einsum("ijklm,ailm->ajk", self.strided_img, self.kernel)
#         print("=================self.conv_img=================")
#         print(self.conv_img)
#         print("=================self.bias=================")
#         print(self.bias)

        self.conv_img += self.bias.reshape(len(self.bias),1, 1)
    
#         print("=================self.conv_img=================")
#         print(self.conv_img)
            
        self.output_img = self.act_func(self.conv_img)               # Through activation function
        self.d_func_img = self.d_act_func(self.conv_img)             # Through derivative of activation function
#         print("=================self.input_img=================")
#         print(self.input_img[0])
#         print("=================self.strided_img=================")
#         print(self.strided_img[0][13])
#         print("=================self.kernel=================")
#         print(self.kernel[0])
#         print("=================self.conv_img=================")
#         print(self.conv_img)
#         print("=================self.output_img=================")
#         print(self.output_img)
    
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        self.pass_error = np.zeros((self.input_num, self.input_size, self.input_size), dtype=np.float)                 # delta_b
        
        # Original strides
        inv_shape = self.pass_error.shape
        inv_stride = self.pass_error.strides
        # Change the pass error to self.strided_img
        self.pass_error = np.lib.stride_tricks.as_strided(self.pass_error, shape=self.stride_shape, strides=self.strides)
        
        self.delta_w = np.zeros((self.filter_num,self.input_num,self.filter_size,self.filter_size), dtype=np.float)  
        self.error = self.d_func_img * self.bp_vec
#         print(self.bp_vec)
        pass_error_tmp = np.einsum("aijk,alm->ilmjk", self.kernel, self.error)
        
        # Adjust weight
        self.delta_w = np.einsum("ijklm,ajk->ailm", self.strided_img, self.error)        
        self.kernel -= lr_rate * self.delta_w 
        
        # Adjust bias
        self.delta_bias = np.einsum("ijk->i", self.error)
        self.bias -= lr_rate * self.delta_bias
        
#         print("=================self.d_func_img=================")
#         print(self.d_func_img)
#         print("=================self.bp_vec=================")
#         print(self.bp_vec)
#         print("=================self.strided_img=================")
#         print(self.strided_img)
#         print("=================self.error=================")
#         print(self.error)
#         print("=================self.delta_w=================")
#         print(self.delta_w)
#         print("=================self.delta_bias=================")
#         print(self.delta_bias)
        
        # Calculate pass error
        
        for i in range(len(pass_error_tmp[0])):
            for j in range(len(pass_error_tmp[0][0])):
        #         print(A[:,i,j], pass_error_tmp[:,i,j])
                self.pass_error[:,i,j] += pass_error_tmp[:,i,j]
#         for img_h in range(len(self.error[0])):
#             for img_w in range(len(self.error[0][0])):
#                 left_corner_h = img_h * self.stride
#                 left_corner_w = img_w * self.stride
                
#                 for feature in range(len(self.error)):
#                     # error pass to previous layer
#                     self.pass_error[:,int(left_corner_h):int(left_corner_h+self.filter_size),int(left_corner_w):int(left_corner_w+self.filter_size)] += self.kernel[feature] * self.error[feature][img_h][img_w]
        
        self.pass_error = np.lib.stride_tricks.as_strided(self.pass_error, shape=inv_shape, strides=inv_stride)       # Cut the image with kernel size
        if not self.is_first:
            self.last_layer.pass_bp(self.pass_error)
                    
    
    def extract_value(self):
        self.input_img = self.last_layer.get_output()
        return self.input_img
    
    def get_output(self):
        return self.output_img.copy()
    
    def get_output_size(self):
        return self.output_size
    
    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_img = x.copy()
        
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()

In [3]:
'''
Pooling class using no padding

param last_layer - point to last layer, which pass the value over
param input_size - input feature maps/images size
param input_num - numbers of input feature maps/images 
param filter_pattern - pattern of the filter
param stride - moving step of the kernel function
'''
class AvgPooling():
    def __init__(self, last_layer, input_size, input_num, filter_size, stride, is_first):
        self.last_layer = last_layer
        self.input_size = int(input_size)
        self.input_num = int(input_num)
        self.filter_size = int(filter_size)
        self.filter_pattern = np.full((int(input_num), int(input_num), int(filter_size), int(filter_size)), 1/(filter_size * filter_size))
        self.d_filter_pattern = np.full((int(input_num), int(filter_size), int(filter_size)), 1/(filter_size * filter_size))
        self.stride = stride
        self.is_first = is_first
        
        # Check if the parameters of input size, filter size, stride is legel
        self.output_size = (input_size + stride - self.filter_size) / stride
        if(self.output_size%1 != 0):
            print("Invalid ! Please check your parameters");
            return -1
        
        self.input_img = 0
        self.stride_shape = (int(input_num),int(self.output_size),int(self.output_size),int(filter_size),int(filter_size))
        self.strides = (int(input_size*input_size*8), int(input_size*stride*8), int(stride*8), int(input_size*8), 8)
        self.conv_img = 0
        self.output_img = 0
        self.d_func_img = 0
        
        self.pass_error = 0      # Error passed to previous layer
        self.error = 0           # input error * derivative of activation function
        
    def forwrad_pass(self):
        if not self.is_first:
            self.extract_value()
        strided_img = np.lib.stride_tricks.as_strided(self.input_img, shape=self.stride_shape, strides=self.strides)       # Cut the image with kernel size
        
        # Convolution operations
        self.output_img = np.einsum("ijklm,ailm->ajk", strided_img, self.filter_pattern)
        ## In max pooling, need to record last max value position
    
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        self.pass_error = np.zeros((self.input_num, self.input_size, self.input_size), dtype=np.float)                 # delta_b
#         self.error = self.d_filter_pattern * self.bp_vec                                                # error
        self.error = self.bp_vec                                                # error
#         print(self.bp_vec  )
        for img_h in range(len(self.error[0])):
            for img_w in range(len(self.error[0][0])):
                left_corner_h = img_h * self.stride
                left_corner_w = img_w * self.stride
                for feature in range(len(self.error)):
                    # error pass to previous layer
#                     print(self.pass_error[:,int(left_corner_h):int(left_corner_h+filter_size),int(left_corner_w):int(left_corner_w+filter_size)])
                    self.pass_error[:,int(left_corner_h):int(left_corner_h+self.filter_size),int(left_corner_w):int(left_corner_w+self.filter_size)] += self.d_filter_pattern * self.bp_vec[feature][img_h][img_w]
        if not self.is_first:
            self.last_layer.pass_bp(self.pass_error)
        
    def extract_value(self):
        self.input_img = self.last_layer.get_output()
        return self.input_img
    
    def get_output(self):
        return self.output_img.copy()
    
    def get_output_size(self):
        return self.output_size
    
    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_img = x.copy()
        
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()

In [4]:
'''
Flattening class using no padding

param last_layer - point to last layer, which pass the value over
param input_size - input feature maps/images size
param input_num - numbers of input feature maps/images
'''
class Flattening():
    def __init__(self, last_layer, input_size, input_num, is_first):
        self.last_layer = last_layer
        self.input_size = int(input_size)
        self.input_num = int(input_num)
        self.is_first = is_first
        
    def forwrad_pass(self):
        if not self.is_first:
            self.extract_value()
        self.output_img = self.input_img.reshape(int(self.input_num*self.input_size*self.input_size))
        
        
    def extract_value(self):
        self.input_img = self.last_layer.get_output()
        return self.input_img
    
    def get_output(self):
        return self.output_img.copy()    
    
    '''
    Set input variable, used for first layer which recieve input value
    @param x - input value for the network
    '''
    def set_input(self, x):
        self.input_img = x.copy()
    
    def get_node_num(self):
        self.neuron_num = self.input_num*self.input_size*self.input_size
        return self.neuron_num
    
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()
        
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        if not self.is_first:
            self.pass_error = self.bp_vec.reshape(int(self.input_num), int(self.input_size), int(self.input_size))
            self.last_layer.pass_bp(self.pass_error)

In [5]:
#@title
'''
Activation function for the network
'''
def test_act_func(x):
    return x*11

'''
ReLU
'''
def ReLU(x):
    x[x<=0] = 0
    return x.copy()

'''
Sigmoid
'''
def Sigmoid(x):
    return 1/(1+np.exp(-x))


In [6]:
#@title
'''
Diviation of the activation function for the network
'''
def d_test_act_func(x):
    return x+2

'''
Diviation of ReLU
'''
def d_ReLU(x):
    x[x > 0] = 1
    x[x < 0] = 0
    return x.copy()

'''
Diviation of Sigmoid
'''
def d_Sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s * (1 - s)